# Cox生存分析

* `mydir`：自己的数据
* `ostime_column`: 数据对应的生存时间，不一定非的是OST，也可以是DST、FST等。
* `os`：生存状态，不一定非的是OS，也可以是DS、FS等。

In [ ]:
from lifelines import CoxPHFitter
import pandas as pd
from onekey_algo import OnekeyDS as okds
from sklearn.model_selection import train_test_split
from onekey_algo import get_param_in_cwd

mydir = get_param_in_cwd('survival_file', None)
ostime_column = get_param_in_cwd('duration_col', 'duration')
os_column = get_param_in_cwd('event_col', 'event')
data = pd.read_csv(mydir)

# 这个地方需要使用自己划分好的数据集
train_data = data[data['group'] == 'train']
train_data = train_data.drop(['ID', 'group'], axis=1)
train_data

## Cox概览

所有Cox回归的必要数据，主要关注的数据有3个
1. `Concordance`: c-index
2. `exp(coef)`: 每个特征对应的HR，同时也有期对应的95%分位数。
3. `p`: 表示特征是否显著。

In [ ]:
cph = CoxPHFitter()
cph.fit(train_data, duration_col=ostime_column, event_col=os_column)

cph.print_summary()

#### 输出每个特征的HR

In [ ]:
import matplotlib.pyplot as plt

cph.plot()
plt.show()

# 绘制Nomogram

要指定绘制nomo图使用到的列，在columns变量。

**注意**：survs表示时间分段，需要根据自己的数据情况划分，如果是时间是天数，则3年、5年生存率对应的survs参数为`survs=[3*365, 5*365]`

In [ ]:
from onekey_algo.custom.components import nomogram

nomogram.nomogram(train_data, duration=ostime_column, result=os_column, 
                  columns=[c for c in train_data if c not in [ostime_column, os_column]],
                  survs=[36, 60], surv_names=['3 year survival','5 year survival'], with_r=False,
                  x_range='0.1,0.25, 0.5,0.75,0.9', height=4000, width=6000)

# KM 曲线

根据HR进行分组，计算KM以及log ranktest

In [ ]:
from lifelines import CoxPHFitter
from lifelines.statistics import logrank_test
from lifelines import KaplanMeierFitter
from lifelines.plotting import add_at_risk_counts

test_data = data[data['group'] == 'test']
c_index = cph.score(test_data[[c for c in test_data.columns if c != 'ID']], scoring_method="concordance_index")
y_pred = cph.predict_partial_hazard(test_data[[c for c in test_data.columns if c != 'ID']])
cox_data = pd.concat([test_data, y_pred], axis=1)

cox_data['HR'] = cox_data[0] > 1
dem = (cox_data["HR"] == True)

results = logrank_test(cox_data[ostime_column][dem], cox_data[ostime_column][~dem], 
                       event_observed_A=cox_data[os_column][dem], event_observed_B=cox_data[os_column][~dem])
p_value = results.p_value
kmf = KaplanMeierFitter()
plt.title(f"C-index:{c_index:.4f}, p_value={p_value}")
# 分成2组之后绘制KM曲线
if sum(dem):
    kmf_high = KaplanMeierFitter()
    kmf_high.fit(cox_data[ostime_column][dem], event_observed=cox_data[os_column][dem], label="High Rish")
    kmf_high.plot_survival_function(color='r')
if sum(~dem):
    kmf_low = KaplanMeierFitter()
    kmf_low.fit(cox_data[ostime_column][~dem], event_observed=cox_data[os_column][~dem], label="Low Risk")
    kmf_low.plot_survival_function(color='g')

add_at_risk_counts(kmf_high, kmf_low, rows_to_show=['At risk'])
plt.savefig(f'KM.svg', bbox_inches='tight')
plt.show()

# 根据先验分组

根据先验特征进行分组，计算KM

In [ ]:
cph.plot_partial_effects_on_outcome(covariates='drink', values=[0, 1], cmap='coolwarm')
plt.xlim(20, 83)
plt.show()